In [3]:
import random
from typing import Optional, List, Dict, Any, TYPE_CHECKING

# =========================================================
# 1. CORE CLASSES (CONSOLIDATED FROM core/ directory)
# =========================================================

# --- Move Class (from core/move.py) ---
class Move:
    def __init__(self, name: str, power: int, type: str, category: str = "physical"):
        self.name = name
        self.power = power
        self.type = type
        self.category = category

    def __repr__(self):
        return self.name

# --- Pokemon Classes (from core/pokemon.py) ---
class PokemonSpecies:
    def __init__(self, name: str, type1: str, base_stats: Dict[str, int]):
        self.name = name
        self.type1 = type1
        self.base_stats = base_stats

class PokemonInstance:
    """Represents an individual Pokémon in battle."""
    def __init__(self, species: PokemonSpecies, level: int = 50):
        self.species = species
        self.level = level
        self.stats = self._calculate_stats()
        self.current_hp = self.stats['hp']
        self.moves: List[Move] = []
        self.ability = self # Simple stub

    def _calculate_stats(self) -> Dict[str, int]:
        # Simple, non-standard stat calculation for stub purposes
        stats = {k: v + self.level * 2 for k, v in self.species.base_stats.items()}
        stats['hp'] = self.species.base_stats['hp'] * 3
        return stats

    def get_stat(self, stat_name: str) -> int:
        """Retrieves a stat value (e.g., 'speed')."""
        return self.stats.get(stat_name.lower(), 0)

    def calculate_damage(self, move: Move, defender: 'PokemonInstance', battle_state: 'BattleState') -> float:
        """Stub damage calculation: Base power + Random variance."""
        base_damage = move.power * (self.level / 100.0)
        # Random factor (85% to 100% damage)
        damage_variance = random.uniform(0.85, 1.0)
        return base_damage * damage_variance

    def take_damage(self, damage: float):
        """Reduces current HP."""
        self.current_hp = max(0, self.current_hp - int(damage))

    def fainted(self) -> bool:
        """Checks if the Pokémon has fainted."""
        return self.current_hp <= 0

    def __repr__(self):
        return f"{self.species.name} (Lvl {self.level}) - {self.current_hp}/{self.stats['hp']} HP"

# --- Action Class (from core/action.py) ---
class Action:
    """Encapsulates a single action chosen by a player or AI."""
    def __init__(self, pokemon: PokemonInstance, move: Move):
        self.pokemon = pokemon
        self.move = move

    def __repr__(self):
        return f"Action: {self.pokemon.species.name} uses {self.move.name}"

# --- BattleState Class (from core/battle_state.py) ---
class BattleState:
    """
    Holds the current state of a Pokémon battle.
    Tracks weather, turn order, field effects, and player Pokémon.
    """

    def __init__(self, pokemon1: PokemonInstance, pokemon2: PokemonInstance):
        self.pokemon1 = pokemon1
        self.pokemon2 = pokemon2
        self.turn = 1
        self.weather: Optional[str] = None
        self.field_effects = {}
        self.active_pokemon = [pokemon1, pokemon2]
        self.last_move: Optional[Move] = None

    def set_weather(self, weather: Optional[str]):
        """Change weather and notify both Pokémon abilities."""
        self.weather = weather
        print(f"The weather changed to {weather or 'clear'}!")
        for p in self.active_pokemon:
            if hasattr(p.ability, "on_weather_change"):
                p.ability.on_weather_change(p, weather)

    def next_turn(self):
        """Increment turn counter and apply end-of-turn effects."""
        print(f"\n--- Turn {self.turn} ends ---")
        for p in self.active_pokemon:
            if hasattr(p.ability, "on_end_turn"):
                p.ability.on_end_turn(p)
        self.turn += 1

    def get_turn_order(self) -> List[PokemonInstance]:
        """Return Pokémon in order of who moves first, based on speed."""
        p1_speed = self.pokemon1.get_stat("speed")
        p2_speed = self.pokemon2.get_stat("speed")
        if p1_speed == p2_speed:
            return random.sample(self.active_pokemon, 2)
        # Sort by speed descending
        return sorted(self.active_pokemon, key=lambda p: p.get_stat("speed"), reverse=True)

    def execute_move(self, attacker: PokemonInstance, defender: PokemonInstance, move: Move):
        """Handles damage, type effectiveness, and applying effects."""

        if attacker.fainted():
            print(f"{attacker.species.name} is unable to move (fainted).")
            return

        print(f"\n{attacker.species.name} used {move.name}!")

        damage = attacker.calculate_damage(move, defender, battle_state=self)
        defender.take_damage(damage)

        self.last_move = move

        print(f"It dealt {int(damage)} damage! {defender.species.name} has {defender.current_hp}/{defender.stats['hp']} HP left.")

        if defender.fainted():
            print(f"{defender.species.name} fainted!")

    def simulate_turn(self, action1: Action, action2: Action):
        """Simulates a full turn using two Action objects."""
        action_map = {
            action1.pokemon: action1,
            action2.pokemon: action2
        }

        order = self.get_turn_order()

        print("\n--- Battle Turn Start ---")

        for pokemon in order:
            action = action_map.get(pokemon)

            # Identify the defender
            defender = self.pokemon2 if pokemon == self.pokemon1 else self.pokemon1

            # Check if the target has fainted from a previous move this turn
            if defender.fainted():
                print(f"{pokemon.species.name} has no target, the battle is over!")
                break

            # Execute the move
            self.execute_move(pokemon, defender, action.move)

        self.next_turn()


# =========================================================
# 2. MAIN BATTLE SIMULATION LOGIC
# =========================================================

# --- Setup Data ---
tackle = Move(name="Tackle", power=40, type="Normal")
flamethrower = Move(name="Flamethrower", power=90, type="Fire")
water_gun = Move(name="Water Gun", power=40, type="Water")
quick_attack = Move(name="Quick Attack", power=40, type="Normal")

species_charizard = PokemonSpecies(
    name="Charizard",
    type1="Fire",
    base_stats={"hp": 78, "attack": 84, "defense": 78, "speed": 100}
)
species_blastoise = PokemonSpecies(
    name="Blastoise",
    type1="Water",
    base_stats={"hp": 79, "attack": 83, "defense": 100, "speed": 78}
)

# --- Initialize Pokémon ---
player_pokemon = PokemonInstance(species_charizard, level=50)
player_pokemon.moves = [flamethrower, quick_attack]
ai_pokemon = PokemonInstance(species_blastoise, level=50)
ai_pokemon.moves = [water_gun, tackle]

# --- AI Logic Helper ---
def choose_ai_action(ai_pkmn: PokemonInstance, opponent_pkmn: PokemonInstance) -> Action:
    """Simple AI: selects a move randomly."""
    chosen_move = random.choice(ai_pkmn.moves)
    print(f"AI ({ai_pkmn.species.name}) chose {chosen_move.name}.")
    return Action(ai_pkmn, chosen_move)

# --- Player Logic Helper (Mocking User Input) ---
def choose_player_action(player_pkmn: PokemonInstance) -> Action:
    """Mocks player input for demonstration."""
    print(f"\n{player_pkmn.species.name}'s moves:")
    for i, move in enumerate(player_pkmn.moves):
        print(f"[{i+1}] {move.name} (Power: {move.power}, Type: {move.type})")

    # Mocking a player choice: choosing move 1 (Flamethrower)
    player_choice_index = 0
    chosen_move = player_pkmn.moves[player_choice_index]

    print(f"\nPlayer chose {chosen_move.name} (Move {player_choice_index + 1}).")
    return Action(player_pkmn, chosen_move)


if __name__ == "__main__":
    print("="*50)
    print("       POKÉMON BATTLE SIMULATION START        ")
    print("="*50)

    # 1. Initialize Battle
    battle_state = BattleState(player_pokemon, ai_pokemon)
    print(f"Battle started: {player_pokemon.species.name} (Speed: {player_pokemon.get_stat('speed')}) vs {ai_pokemon.species.name} (Speed: {ai_pokemon.get_stat('speed')})")
    print(f"Initial HP: {player_pokemon.current_hp} / {ai_pokemon.current_hp}")


    # --- Turn 1 ---
    print("\n" + "="*30)
    print("             TURN 1            ")
    print("="*30)

    # 2. Player chooses action
    player_action = choose_player_action(player_pokemon)

    # 3. AI chooses action
    ai_action = choose_ai_action(ai_pokemon, player_pokemon)

    # 4. Execute the turn
    battle_state.simulate_turn(player_action, ai_action)

    # --- Turn 2 (Example of ongoing battle) ---
    if not player_pokemon.fainted() and not ai_pokemon.fainted():
        print("\n" + "="*30)
        print("             TURN 2            ")
        print("="*30)

        # Player uses Quick Attack this time (move 2)
        player_action_2 = Action(player_pokemon, player_pokemon.moves[1])

        # AI uses a random move again
        ai_action_2 = choose_ai_action(ai_pokemon, player_pokemon)

        battle_state.simulate_turn(player_action_2, ai_action_2)


       POKÉMON BATTLE SIMULATION START        
Battle started: Charizard (Speed: 200) vs Blastoise (Speed: 178)
Initial HP: 234 / 237

             TURN 1            

Charizard's moves:
[1] Flamethrower (Power: 90, Type: Fire)
[2] Quick Attack (Power: 40, Type: Normal)

Player chose Flamethrower (Move 1).
AI (Blastoise) chose Tackle.

--- Battle Turn Start ---

Charizard used Flamethrower!
It dealt 38 damage! Blastoise has 199/237 HP left.

Blastoise used Tackle!
It dealt 18 damage! Charizard has 216/234 HP left.

--- Turn 1 ends ---

             TURN 2            
AI (Blastoise) chose Tackle.

--- Battle Turn Start ---

Charizard used Quick Attack!
It dealt 18 damage! Blastoise has 181/237 HP left.

Blastoise used Tackle!
It dealt 19 damage! Charizard has 197/234 HP left.

--- Turn 2 ends ---
